# <center><font color='Red' text align='center'>Digit recognition script</font></center>

In this notebook I will cover how the script is used and what I have done to try to get the accuracy as high as I can.

This is the main code from the script file and it is currently working at a 97% accuracy. 

In [22]:
import gzip
with gzip.open('data/t10k-images-idx3-ubyte.gz', 'rb') as f:
        file_content = f.read()

# Import keras.
import keras as kr

# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=600, activation='linear', input_dim=784))
model.add(kr.layers.Dense(units=400, activation='relu'))
# Add a three neuron output layer.
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with gzip.open('data/train-images-idx3-ubyte.gz', 'rb') as f:
    train_img = f.read()

with gzip.open('data/train-labels-idx1-ubyte.gz', 'rb') as f:
    train_lbl = f.read()
    
import numpy as np
        
train_img = ~np.array(list(train_img[16:])).reshape(60000, 28, 28).astype(np.uint8) / 255.0
train_lbl =  np.array(list(train_lbl[ 8:])).astype(np.uint8)

inputs = train_img.reshape(60000, 784)

 # For encoding categorical variables.
import sklearn.preprocessing as pre

encoder = pre.LabelBinarizer()
encoder.fit(train_lbl)
outputs = encoder.transform(train_lbl)

model.fit(inputs, outputs, epochs=15, batch_size=100)

Epoch 1/15
60000/60000 [==============================] - 12s 192us/step - loss: 0.5692 - acc: 0.8339
Epoch 2/15
60000/60000 [==============================] - 11s 179us/step - loss: 0.2518 - acc: 0.9229
Epoch 3/15
60000/60000 [==============================] - 11s 178us/step - loss: 0.1801 - acc: 0.9447
Epoch 4/15
60000/60000 [==============================] - 11s 178us/step - loss: 0.1595 - acc: 0.9501
Epoch 5/15
60000/60000 [==============================] - 11s 178us/step - loss: 0.1454 - acc: 0.9553
Epoch 6/15
60000/60000 [==============================] - 11s 189us/step - loss: 0.1377 - acc: 0.9570
Epoch 7/15
60000/60000 [==============================] - 11s 191us/step - loss: 0.1214 - acc: 0.9624
Epoch 8/15
60000/60000 [==============================] - 11s 190us/step - loss: 0.1136 - acc: 0.9649
Epoch 9/15
60000/60000 [==============================] - 11s 191us/step - loss: 0.1131 - acc: 0.9656
Epoch 10/15
60000/60000 [==============================] - 12s 192us/step - loss: 

##### Trying to improve accuracy

There are a few condations we can change to try to improve accuracy. 

Looking at the following line :

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


we can see using https://keras.io/optimizers/ that there are a number of different optimizers available so how do we find the best one.

if we change the optimizer to Nadam we can see the network performs worse, much worse.

In [23]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=600, activation='linear', input_dim=784))
model.add(kr.layers.Dense(units=400, activation='relu'))
# Add a three neuron output layer.
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

with gzip.open('data/train-images-idx3-ubyte.gz', 'rb') as f:
    train_img = f.read()

with gzip.open('data/train-labels-idx1-ubyte.gz', 'rb') as f:
    train_lbl = f.read()
    
import numpy as np
        
train_img = ~np.array(list(train_img[16:])).reshape(60000, 28, 28).astype(np.uint8) / 255.0
train_lbl =  np.array(list(train_lbl[ 8:])).astype(np.uint8)

inputs = train_img.reshape(60000, 784)

encoder = pre.LabelBinarizer()
encoder.fit(train_lbl)
outputs = encoder.transform(train_lbl)

model.fit(inputs, outputs, epochs=5, batch_size=100)

Epoch 1/5
60000/60000 [==============================] - 14s 232us/step - loss: 14.3056 - acc: 0.1000
Epoch 2/5
60000/60000 [==============================] - 13s 211us/step - loss: 14.5463 - acc: 0.0975
Epoch 3/5
60000/60000 [==============================] - 13s 211us/step - loss: 14.5463 - acc: 0.0975
Epoch 4/5
60000/60000 [==============================] - 13s 209us/step - loss: 14.5463 - acc: 0.0975
Epoch 5/5
60000/60000 [==============================] - 13s 209us/step - loss: 14.5463 - acc: 0.0975


The Neural Network is actually most effective on it's first run and then remains as a constant level after. It's learning nothing new from each run and this is because of the optimizer Nadam. It is obviously not fit for purpose.

###### Continuing to read through the optimizers documentation we can see adamax. 

If we change the optimizer to Adamax we can see the network performs worse than adam at the start but improves much better over each Epoch/run. The learning rate is better and running both Adam and adamax for 15 Epochs sees it overtake adam optimizer for accuracy. 

In [24]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=600, activation='linear', input_dim=784))
model.add(kr.layers.Dense(units=400, activation='relu'))
# Add a three neuron output layer.
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])

with gzip.open('data/train-images-idx3-ubyte.gz', 'rb') as f:
    train_img = f.read()

with gzip.open('data/train-labels-idx1-ubyte.gz', 'rb') as f:
    train_lbl = f.read()
    
import numpy as np
        
train_img = ~np.array(list(train_img[16:])).reshape(60000, 28, 28).astype(np.uint8) / 255.0
train_lbl =  np.array(list(train_lbl[ 8:])).astype(np.uint8)

inputs = train_img.reshape(60000, 784)

encoder = pre.LabelBinarizer()
encoder.fit(train_lbl)
outputs = encoder.transform(train_lbl)

model.fit(inputs, outputs, epochs=15, batch_size=100)

Epoch 1/15
60000/60000 [==============================] - 12s 199us/step - loss: 2.3980 - acc: 0.7290
Epoch 2/15
60000/60000 [==============================] - 11s 185us/step - loss: 0.7115 - acc: 0.8681
Epoch 3/15
60000/60000 [==============================] - 11s 185us/step - loss: 0.2584 - acc: 0.9210
Epoch 4/15
60000/60000 [==============================] - 11s 189us/step - loss: 0.1923 - acc: 0.9413
Epoch 5/15
60000/60000 [==============================] - 11s 186us/step - loss: 0.1572 - acc: 0.9510
Epoch 6/15
60000/60000 [==============================] - 11s 190us/step - loss: 0.1374 - acc: 0.9581
Epoch 7/15
60000/60000 [==============================] - 11s 182us/step - loss: 0.1183 - acc: 0.9628
Epoch 8/15
60000/60000 [==============================] - 11s 185us/step - loss: 0.1058 - acc: 0.9667
Epoch 9/15
60000/60000 [==============================] - 11s 188us/step - loss: 0.0975 - acc: 0.9701
Epoch 10/15
60000/60000 [==============================] - 11s 188us/step - loss: 

So because of it's improved peformance Adamax is the better optmizer over the long run. Used in the script for 50 epochs each the Adam optimizer topped out around 98%, Adamax was able to get the accuracy to 99.5%. ON a 30 epoch run it will get an accuracy of 99%.

![title](https://raw.githubusercontent.com/DuffyTJ89/EmerTechAssignment/master/images/DigitRec/ScriptOutcome.PNG?token=AVaH05ZmDNHEL4wOJ6Mevny5VBR-_JEnks5cCtMBwA%3D%3D)

End